<a href="https://colab.research.google.com/github/tyllersun/NTHU_EEclass_TAhelper/blob/main/%E5%BA%A7%E4%BD%8D%E8%A1%A8%E7%94%A2%E7%94%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
def extract_student_info(file_path):
    """
    Extract student names and IDs from an Excel file.

    Parameters:
        file_path (str): Path to the Excel file.

    Returns:
        DataFrame: A DataFrame containing student names and IDs.
    """
    # Load the Excel file, using the second row (index 1) as the header
    df = pd.read_excel(file_path, header=1)

    # Extract the columns with student names and IDs
    students_info = df[['姓名', '帳號']]

    return students_info

# Extract student info and display the first few rows
students_info = extract_student_info("/content/統計學二Statistics II_學員名單.xlsx")
students_info.head()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np

# 下载中文字體
!wget -O SourceHanSansSC-Regular.otf "https://github.com/adobe-fonts/source-han-sans/raw/release/OTF/SimplifiedChinese/SourceHanSansSC-Regular.otf"
prop = fm.FontProperties(fname="SourceHanSansSC-Regular.otf")

def create_seating_chart_pdf(students_df, width, height, aisles, seed=None, filename="seating_chart.pdf"):
    """
    創建教室座位表並標記走道位置，然後保存為PDF。

    參數:
      students_df (DataFrame): 學生名單的dataframe，包含"姓名"和"帳號"兩列
      width (int): 教室的寬度（座位數）
      height (int): 教室的高度（座位數）
      aisles (list of int): 走道所在的列號
      seed (int, optional): 隨機數生成器的種子，用於確保每次打亂的順序相同
      filename (str): 保存的PDF文件名

    返回:
      None
    """
    np.random.seed(seed)
    fig_height = 210 / 25.4
    fig_width = 297 / 25.4
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))


    x_scale = fig_width / width
    y_scale = fig_height / (height + 2)
    scale = min(x_scale, y_scale)

    ax.set_xlim(0, width * scale)
    ax.set_ylim(0, (height + 2) * scale)
    ax.axis("off")


    shuffled_students = students_df.sample(frac=1).reset_index(drop=True)

    student_counter = 0
    for row in reversed(range(height)):
        for col in range(width):
            if col + 1 in aisles:
                color = 'grey'
                student_name = ''
                student_id = ''
            else:
                if student_counter < len(shuffled_students) and (col % 2 == 0):
                    color = 'white'
                    student_name = shuffled_students["姓名"][student_counter]
                    student_id = shuffled_students["帳號"][student_counter]
                    student_counter += 1
                else:
                    color = 'white'
                    student_name = ''
                    student_id = ''
            rect = patches.Rectangle((col * scale, (height - row + 1) * scale), scale, scale, linewidth=1, edgecolor='black', facecolor=color)
            ax.add_patch(rect)
            ax.text(col * scale + scale/2, (height - row + 1.7) * scale, student_name, color='black', ha='center', va='center', fontproperties=prop, fontsize=8)
            ax.text(col * scale + scale/2, (height - row + 1.3) * scale, student_id, color='black', ha='center', va='center', fontproperties=prop, fontsize=8)

    # 畫講臺
    podium_width = width * 0.6 * scale
    podium_start = (width - podium_width / scale) / 2 * scale
    rect = patches.Rectangle((podium_start, 0.5 * scale), podium_width, scale, linewidth=1, edgecolor='black', facecolor='white')
    ax.add_patch(rect)
    ax.text(width/2 * scale, scale, '講台', color='black', ha='center', va='center', fontproperties=prop)

    with PdfPages(filename) as pdf:
        pdf.savefig(fig, bbox_inches='tight')

# 補考教室
width = 5
height = 7
aisles = []

# 正課教室
width = 16
height = 8
aisles = [5, 12]  # 第3列和第7列是走道
create_seating_chart_pdf(students_info, width, height, aisles, seed=429)
